In [31]:
import requests
import pandas as pd
import numpy as np

In [32]:
api_key = "TJI6M4BFN9J1591S49US8UZ7MSZFX5JK8Y"
address = "0xdbf5e9c5206d0db70a90108bf936da60221dc080"
contract_address = "0x92D6C1e31e14520e676a687F0a93788B716BEff5" ## DYDX
#drawl Token Transfer Events' by Address from etherscan api 
#https://docs.etherscan.io/api-endpoints/accounts#get-a-list-of-erc20-token-transfer-events-by-address
url = f"https://api.etherscan.io/api?module=account&action=tokentx&contractaddress={contract_address}&address={address}&page=1&offset=100&sort=desc&apikey={api_key}"
response = requests.get(url)
data = response.json()
transactions = data["result"]

#select relevant columns 
df_token_transfer = pd.DataFrame(transactions)
df_token_transfer = df_token_transfer[["timeStamp", "hash", "from","to","tokenName","tokenSymbol","value","tokenDecimal"]]

#convert datatype
df_token_transfer["tokenDecimal"] = np.int64(df_token_transfer["tokenDecimal"])
df_token_transfer["amount"] = df_token_transfer["value"].astype(float) / pow(10,df_token_transfer["tokenDecimal"])
df_token_transfer["amount"] = df_token_transfer["amount"].astype(int)
df_token_transfer["timeStamp"] = pd.to_datetime(df_token_transfer["timeStamp"], unit="s")

df_token_transfer.head()

,timeStamp,hash,from,to,tokenName,tokenSymbol,value,tokenDecimal,amount
0,2023-03-17 06:59:11,0x704bcfe632831e1812631836103fbb57a7da06295db0...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xc04c8b02633d2de39541f25de0cd941fdb0068fe,dYdX,DYDX,100000000000000000000000,18,99999
1,2023-03-11 00:38:47,0x5220b86c6d8179a6f38cf210e17a6bd74c2072594faf...,0xfcaf9c57c26566f96d23f585950bb1c66e138890,0xdbf5e9c5206d0db70a90108bf936da60221dc080,dYdX,DYDX,373937000000000000000000,18,373937
2,2023-03-03 22:21:47,0xb3a81882007d37f80d5588cd70caf80d9d38f8c74bf1...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,dYdX,DYDX,300000000000000000000000,18,300000
3,2023-03-03 21:29:59,0xb08ce240be379a92a8128fa5739a2d1732e5852f50b5...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xc04c8b02633d2de39541f25de0cd941fdb0068fe,dYdX,DYDX,10000000000000000000,18,10
4,2023-02-28 13:57:47,0xc39085a470b7afa883397f60bbc0f79f86f533749df5...,0x2a9df649f88b9ea0d85a405299ceab227da4c5dd,0xdbf5e9c5206d0db70a90108bf936da60221dc080,dYdX,DYDX,4279156933188489636132810,18,4279156


In [33]:
from_day = int(round(df_token_transfer["timeStamp"].iloc[-1].timestamp()))
to_day = int(round(df_token_transfer["timeStamp"].iloc[0].timestamp()))
currency = 'usd'

# Get historical market data include price, market cap, and 24h volume within a range of timestamp
# https://www.coingecko.com/en/api/documentation
url1 = f"https://api.coingecko.com/api/v3/coins/1/contract/{contract_address}/market_chart/range?vs_currency={currency}&from={from_day}&to={to_day}"
response = requests.get(url1)
data = response.json()

df_token = pd.DataFrame(data)
df_token[["timestamp", "price_value"]] = df_token["prices"].apply(lambda x: pd.Series([x[0], x[1]]))
df_token["timestamp"] = np.int64(df_token["timestamp"])

# Sorting by date
df_token.sort_values(by = ["timestamp"], ascending = False, inplace = True)
df_token["timestamp"] = pd.to_datetime(df_token["timestamp"], unit="ms")

df_token_price = df_token[["timestamp", "price_value"]]
df_token_price

,timestamp,price_value
542,2023-03-17,2.408729
541,2023-03-16,2.278995
540,2023-03-15,2.594745
539,2023-03-14,2.389217
538,2023-03-13,2.257698
...,...,...
4,2021-09-25,12.061405
3,2021-09-24,13.933715
2,2021-09-23,14.660788
1,2021-09-22,12.318168


In [37]:
# calculate time difference
time_diff = []
for timestamp in df_token_transfer['timeStamp']:
    time_diff.append((df_token_price['timestamp'] - timestamp).abs().idxmin())

# get corresponding price from df_token_price
prices = df_token_price.loc[time_diff, 'price_value'].values

# add prices as new column in df_token_transfer
df_token_transfer['price'] = round((prices*df_token_transfer['amount']),2)
df_token_transfer = df_token_transfer[["timeStamp", "hash", "from","to","tokenName","tokenSymbol","amount","price"]]
df_token_transfer.head()

,timeStamp,hash,from,to,tokenName,tokenSymbol,amount,price
0,2023-03-17 06:59:11,0x704bcfe632831e1812631836103fbb57a7da06295db0...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xc04c8b02633d2de39541f25de0cd941fdb0068fe,dYdX,DYDX,99999,240870.47
1,2023-03-11 00:38:47,0x5220b86c6d8179a6f38cf210e17a6bd74c2072594faf...,0xfcaf9c57c26566f96d23f585950bb1c66e138890,0xdbf5e9c5206d0db70a90108bf936da60221dc080,dYdX,DYDX,373937,746859.12
2,2023-03-03 22:21:47,0xb3a81882007d37f80d5588cd70caf80d9d38f8c74bf1...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xb3f923eabaf178fc1bd8e13902fc5c61d3ddef5b,dYdX,DYDX,300000,784441.35
3,2023-03-03 21:29:59,0xb08ce240be379a92a8128fa5739a2d1732e5852f50b5...,0xdbf5e9c5206d0db70a90108bf936da60221dc080,0xc04c8b02633d2de39541f25de0cd941fdb0068fe,dYdX,DYDX,10,26.15
4,2023-02-28 13:57:47,0xc39085a470b7afa883397f60bbc0f79f86f533749df5...,0x2a9df649f88b9ea0d85a405299ceab227da4c5dd,0xdbf5e9c5206d0db70a90108bf936da60221dc080,dYdX,DYDX,4279156,12467927.08
